<a href="https://colab.research.google.com/github/Mphaka-Mankgele/Data_Science_in_High_Energy_Physics/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt

# Define the transform with data augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Function to create data loaders
def create_data_loaders(batch_size):
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = nn.MaxPool2d(2)(x)
        x = nn.ReLU()(self.bn2(self.conv2(x)))
        x = nn.MaxPool2d(2)(x)
        x = x.view(-1, 64 * 8 * 8)
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

probabilities = []
y_true = []
# Function to train the model
def train_model(train_loader, test_loader, learning_rate, batch_size, dropout_rate, num_epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SimpleCNN(dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()  # Clear gradients
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Evaluate on test set
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    y_pred = []
    all_outputs = []  # For ROC curve

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            all_outputs.append(outputs.cpu())  # Collect outputs for ROC
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true.extend(labels.cpu().numpy())  # Add true labels
            y_pred.extend(predicted.cpu().numpy())  # Add predicted labels

    test_accuracy = 100 * correct / total
    f1 = f1_score(y_true, y_pred, average='weighted')  # Calculate F1 score

    # Calculate ROC curve
    all_outputs = torch.cat(all_outputs).numpy()  # Combine all outputs
    # Convert outputs to probabilities using softmax
    probabilities = nn.functional.softmax(torch.tensor(all_outputs), dim=1).numpy()

    # Calculate ROC curve for each class
    num_classes = 10
    roc_auc = {}
    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(np.array(y_true) == i, probabilities[:, i])  # True positive rates
        roc_auc[i] = auc(fpr, tpr)

    return test_accuracy, f1, roc_auc

# Example usage
learning_rate = 0.0001
batch_size = 64
dropout_rate = 0.5
train_loader, test_loader = create_data_loaders(batch_size)
test_accuracy, f1, roc_auc = train_model(train_loader, test_loader, learning_rate, batch_size, dropout_rate)

print(f'Test Accuracy: {test_accuracy:.2f}%, F1 Score: {f1:.4f}')
for class_idx, auc_value in roc_auc.items():
    print(f'Class {class_idx}: ROC AUC = {auc_value:.4f}')

# Optional: Plot ROC curves
plt.figure(figsize=(10, 8))
for i in range(10):
    fpr, tpr, _ = roc_curve(np.array(y_true) == i, probabilities[:, i])
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc='lower right')
plt.show()


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Loss: 1297.9529, Train Accuracy: 39.54%
Epoch [2/100], Loss: 1074.3776, Train Accuracy: 50.28%
Epoch [3/100], Loss: 982.9426, Train Accuracy: 55.05%
Epoch [4/100], Loss: 933.1585, Train Accuracy: 57.28%
Epoch [5/100], Loss: 894.4867, Train Accuracy: 59.22%
Epoch [6/100], Loss: 864.3475, Train Accuracy: 60.72%
Epoch [7/100], Loss: 839.4826, Train Accuracy: 62.23%
Epoch [8/100], Loss: 817.3787, Train Accuracy: 63.13%
Epoch [9/100], Loss: 797.2815, Train Accuracy: 63.94%
Epoch [10/100], Loss: 785.3640, Train Accuracy: 64.39%
Epoch [11/100], Loss: 763.9933, Train Accuracy: 65.42%
Epoch [12/100], Loss: 756.0867, Train Accuracy: 66.06%
Epoch [13/100], Loss: 741.7495, Train Accuracy: 66.65%
Epoch [14/100], Loss: 733.0374, Train Accuracy: 67.12%
Epoch [15/100], Loss: 722.6568, Train Accuracy: 67.58%
Epoch [16/100], Loss: 712.2776, Train Accuracy: 67.88%
Epoch [17/100], Loss: 705.6318, Train Accuracy: 68.